In [ ]:
#TODO chuẩn bị dữ liệu: mỗi văn bản ứng với 1 chunk (chunk có điểm cao nhất từ retrieval step)
cross_encoder_inputs = []
for chunk_data in candidate_chunks:
    cross_encoder_inputs.append( (query, chunk_data['chunk_text']) )

In [ ]:


model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
reranker_model = CrossEncoder(model_name)


scores = reranker_model.predict(cross_encoder_inputs, show_progress_bar=True) # return a numpy array

for i in range(len(candidate_chunks)):
    candidate_chunks[i]['rerank_score'] = scores[i]

print("\n--- Điểm Rerank cho từng ĐOẠN (Chunk) ---")
for chunk in sorted(candidate_chunks, key=lambda x: x['rerank_score'], reverse=True):
    print(f"  Score: {chunk['rerank_score']:.4f} | Nguồn: {chunk['doc_id']} | Text: {chunk['chunk_text'][:50]}...")



In [ ]:
# =============================================================================
# 3. TỔNG HỢP ĐIỂM "MaxP", lấy doc có chunk_score cao nhất
# =============================================================================

doc_scores = defaultdict(lambda: -float('inf'))

for chunk in candidate_chunks:
    doc_id = chunk['doc_id']
    score = chunk['rerank_score']

    if score > doc_scores[doc_id]:
        doc_scores[doc_id] = score

sorted_docs = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)

K = 2

print(f"\n--- KẾT QUẢ TOP-{K} TÀI LIỆU GỐC (CHIẾN LƯỢC MaxP) ---")

for i, (doc_id, max_score) in enumerate(sorted_docs[:K]):
    print(f"**Xếp hạng {i+1}**")
    print(f"  Tài liệu ID: {doc_id}")
    print(f"  Điểm 'MaxP' (điểm chunk cao nhất): {max_score:.4f}")